In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
from fastai.structured import *
from fastai.column_data import *
from fastai.imports import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LassoLars, ARDRegression, PassiveAggressiveRegressor, TheilSenRegressor

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from IPython.display import display
from sklearn import metrics
np.set_printoptions(threshold=50, edgeitems=20)
import matplotlib.pyplot as plt
from sklearn.kernel_ridge import KernelRidge
PATH='data/case_study/'

/home/p_abhijeet666/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/p_abhijeet666/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data=pd.read_csv(f'{PATH}AllData_new.csv')

In [3]:
data.tail()

,D[-],D_dot[1/s],p[-],p_dot[1/s],Mises[MPa],Hydro[MPa],teq[-],eeq[-],V[mm^3],Sig_tri[-],LoadingState
3388,0.248556,1.983663,0.062001,0.116834,284.832377,18.745354,0.061949,0.001537,0.000212,0.066052,7.0
3389,0.248579,1.948299,0.062003,0.116802,284.825419,18.743141,0.061950,0.001537,0.000212,0.066044,7.0
3390,0.248600,1.886919,0.062004,0.110842,284.818513,18.740990,0.061950,0.001537,0.000212,0.066037,7.0
3391,0.248601,1.059331,0.062004,0.135451,284.818233,18.740799,0.061951,0.001537,0.000212,0.066036,7.0
3392,0.248601,1.091343,0.062004,-0.737903,284.817976,18.740666,0.061951,0.001537,0.000212,0.066036,7.0


In [4]:
data.shape

(3393, 11)

In [5]:
cols = list(data.columns.values)
cols

['D[-]',
 'D_dot[1/s]',
 'p[-]',
 'p_dot[1/s]',
 'Mises[MPa]',
 'Hydro[MPa]',
 'teq[-]',
 'eeq[-]',
 'V[mm^3]',
 'Sig_tri[-]',
 'LoadingState']

In [6]:
damage=data.pop('D[-]')

In [7]:
data.insert(0,'D[-]',damage)

In [8]:
data.tail()

,D[-],D_dot[1/s],p[-],p_dot[1/s],Mises[MPa],Hydro[MPa],teq[-],eeq[-],V[mm^3],Sig_tri[-],LoadingState
3388,0.248556,1.983663,0.062001,0.116834,284.832377,18.745354,0.061949,0.001537,0.000212,0.066052,7.0
3389,0.248579,1.948299,0.062003,0.116802,284.825419,18.743141,0.061950,0.001537,0.000212,0.066044,7.0
3390,0.248600,1.886919,0.062004,0.110842,284.818513,18.740990,0.061950,0.001537,0.000212,0.066037,7.0
3391,0.248601,1.059331,0.062004,0.135451,284.818233,18.740799,0.061951,0.001537,0.000212,0.066036,7.0
3392,0.248601,1.091343,0.062004,-0.737903,284.817976,18.740666,0.061951,0.001537,0.000212,0.066036,7.0


In [9]:
#cat_vars = []
contin_vars = ['D[-]',
 'D_dot[1/s]',
 'p[-]',
 'p_dot[1/s]',
 'Mises[MPa]',
 'Hydro[MPa]',
 'teq[-]',
 'eeq[-]',
 'V[mm^3]',
 'Sig_tri[-]',
 'LoadingState']


In [10]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_test = 500  
n_trn = len(data)-n_test
data_train, data_test = split_vals(data, n_trn)

data_train.shape, data_test.shape

((2893, 11), (500, 11))

In [11]:
dep = 'D[-]'
joined_train = data_train[contin_vars].copy()#data_train[cat_vars+contin_vars].copy()
joined_test = data_test[contin_vars].copy() #data_test[cat_vars+contin_vars].copy()
joined_test[dep] = 0
joined_test.tail()

,D[-],D_dot[1/s],p[-],p_dot[1/s],Mises[MPa],Hydro[MPa],teq[-],eeq[-],V[mm^3],Sig_tri[-],LoadingState
3388,0,1.983663,0.062001,0.116834,284.832377,18.745354,0.061949,0.001537,0.000212,0.066052,7.0
3389,0,1.948299,0.062003,0.116802,284.825419,18.743141,0.061950,0.001537,0.000212,0.066044,7.0
3390,0,1.886919,0.062004,0.110842,284.818513,18.740990,0.061950,0.001537,0.000212,0.066037,7.0
3391,0,1.059331,0.062004,0.135451,284.818233,18.740799,0.061951,0.001537,0.000212,0.066036,7.0
3392,0,1.091343,0.062004,-0.737903,284.817976,18.740666,0.061951,0.001537,0.000212,0.066036,7.0


In [12]:
#for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()
#apply_cats(joined_test, joined)
for v in contin_vars:
    joined_train[v] = joined_train[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [13]:
df_train, y_train, nas,mapper = proc_df(joined_train,dep, do_scale= True)
df_test, y_test, nas,mapper = proc_df(joined_test,dep, do_scale= True)

In [14]:
df_train.tail()

,D_dot[1/s],p[-],p_dot[1/s],Mises[MPa],Hydro[MPa],teq[-],eeq[-],V[mm^3],Sig_tri[-],LoadingState
2888,-0.046843,1.722374,-0.074571,-0.770446,-0.915174,1.726592,1.738634,1.346114,-1.202803,1.432075
2889,-0.046420,1.722685,-0.074298,-0.771683,-0.914918,1.726921,1.743508,1.347383,-1.199008,1.432075
2890,-0.045663,1.723151,-0.073747,-0.773544,-0.914546,1.727415,1.751009,1.349262,-1.199333,1.432075
2891,-0.044897,1.723501,-0.072967,-0.774933,-0.914268,1.727787,1.756781,1.350634,-1.199549,1.432075
2892,-0.044172,1.723764,-0.072231,-0.775976,-0.914059,1.728065,1.761213,1.351700,-1.199694,1.432075


In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

In [15]:
m_LinearRegression= LinearRegression(normalize=True)
m_LinearRegression.fit(df_train, y_train)
rmse(m_LinearRegression.predict(df_test), y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [21]:
m_BayesianRidge=  BayesianRidge()
m_BayesianRidge.fit(df_train,y_train)
rmse(m_BayesianRidge.predict(df_test), y_test)

0.24129387099834376

In [22]:
m_LassoLars=  LassoLars()
m_LassoLars.fit(df_train,y_train)
rmse(m_LassoLars.predict(df_test), y_test)

0.17236094176769257

In [30]:
m_ARDRegression= ARDRegression()
m_ARDRegression.fit(df_train,y_train)
rmse(m_ARDRegression.predict(df_test), y_test)

0.2727385582228025

In [28]:
m_PassiveAggressiveRegressor= PassiveAggressiveRegressor()
m_PassiveAggressiveRegressor.fit(df_train,y_train)
rmse(m_PassiveAggressiveRegressor.predict(df_test), y_test)

/home/p_abhijeet666/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.17596622725149766

In [29]:
m_TheilSenRegressor= TheilSenRegressor()
m_TheilSenRegressor.fit(df_train,y_train)
rmse(m_TheilSenRegressor.predict(df_test), y_test)

0.33273320393712935